# Overview

This notebook uses perch-hoplite to compute and save embeddings for set of audio files using a pre-trained model. This is the first step in the agile modeling process. If the data you wish to search and classify is already embedded with a pre-trained model into a perch-hoplite database, then proceed to the step 2 colab notebook ([2_agile_modeling_v2.ipynb](https://github.com/google-research/perch-hoplite/blob/main/perch_hoplite/agile/2_agile_modeling_v2.ipynb)).

## [Optional] perch-hoplite installation for hosted runtimes

If you have not already installed perch-hoplite (particularly if you are using a hosted Colab runtime), make sure to install perch-hoplite from the Github source to ensure the most recent version is installed. After installation, you will need to restart your runtime before running anything else. Go to the top menu, select "Runtime" then "Restart Session".

In [ ]:
#@title Only run this code if you need to install perch-hoplite
#!pip install git+https://github.com/google-research/perch-hoplite.git

In [1]:
# @title Imports
from etils import epath
from IPython.display import display
import ipywidgets as widgets
import numpy as np
from perch_hoplite.agile import colab_utils
from perch_hoplite.agile import embed
from perch_hoplite.agile import source_info
from perch_hoplite.db import brutalism
from perch_hoplite.db import interface

# Embed the audio data

In [ ]:
# @title Configuration { vertical-output: true }

# @markdown Configure the raw dataset and output location(s).  The format is a mapping from
# @markdown a dataset_name to a (base_path, fileglob) pair.  Note that the file
# @markdown globs are case sensitive.  The dataset name can be anything you want.
#
# @markdown This structure allows you to move your data around without having to
# @markdown re-embed the dataset.  The generated embedding database will be
# @markdown placed in the base path. This allows you to simply swap out
# @markdown the base path here if you ever move your dataset.

# @markdown By default we only process one dataset at a time.  Re-run this entire notebook
# @markdown once per dataset.

# @markdown For example, we might set dataset_base_path to '/home/me/myproject',
# @markdown and use the glob '\*/\*.wav' if all of the audio files have filepaths
# @markdown like '/home/me/myproject/site_XYZ/audio_ABC.wav' (e.g. audio files are contained in subfolders of the base directory).

# @markdown 1. Create a unique name for the database that will store the embeddings for the target data.
dataset_name = 'mojave_data'  # @param {type:'string'}
# @markdown 2. Input the filepath for the folder that is containing the input audio files.
dataset_base_path = '/workspace/mojave_data'  #@param {type:'string'}
# @markdown 3. Input the file pattern for the audio files within that folder that you want to embed. Some examples for how to input:
# @markdown - All files in the base directory of a specific type (not subdirectories): e.g. `*.wav` (or `*.flac` etc) will generate embeddings for all .wav files (or whichever format) in the dataset_base_path
# @markdown - All files in one level of subdirectories within the base directory: `*/*.flac` will generate embeddings for all .flac files
# @markdown - Single file: `myfile.wav` will only embed the audio from that specific file.
dataset_fileglob = 'BD-*/*.wav'  # @param {type:'string'}

# @markdown 4. [Optional] If saving the embeddings database to a new directory, specify here.
# @markdown Otherwise, leave blank - by default the embeddings database output will be saved within
# @markdown dataset_base_path where the audio is located. You do not need to specify db_path unless you want to maintain multiple
# @markdown distinct embedding databases, or if you would like to save the output
# @markdown in a different folder. If your input audio data is accessed
# @markdown from a public URL, we recommend specifying a separate output directory here.
db_path = ''  # @param {type:'string'}
if not db_path or db_path == 'None':
  db_path = None

# @markdown 5. Choose a supported model to generate embeddings: `perch_8` or `birdnet_v2.3` are most common
# @markdown for birds. Other choices include `surfperch` for coral reefs or
# @markdown `multispecies_whale` for marine mammals.
model_choice = 'perch_8'  #@param['perch_8', 'humpback', 'multispecies_whale', 'surfperch', 'birdnet_V2.3']

# @markdown 6. [Optional] Shard the audio for embeddings. File sharding automatically splits audio files into smaller chunks
# @markdown for creating embeddings. This limits both system and GPU memory usage,
# @markdown especially useful when working with long files (>1 hour).
use_file_sharding = True  # @param {type:'boolean'}
# @markdown If you want to change the length in seconds for the shards, specify here.
shard_length_in_seconds = 60  # @param {type:'number'}

audio_glob = source_info.AudioSourceConfig(
    dataset_name=dataset_name,
    base_path=dataset_base_path,
    file_glob=dataset_fileglob,
    min_audio_len_s=1.0,
    target_sample_rate_hz=-2,
    shard_len_s=float(shard_length_in_seconds) if use_file_sharding else None,
)

configs = colab_utils.load_configs(
    source_info.AudioSources((audio_glob,)),
    db_path,
    model_config_key=model_choice,
    db_key='sqlite_usearch',
)
configs

AgileConfigs(audio_sources_config=AudioSources(audio_globs=(AudioSourceConfig(dataset_name='mojave_data', base_path='/workspace/mojave_data', file_glob='BD-*/*.wav', min_audio_len_s=1.0, target_sample_rate_hz=-2, shard_len_s=60.0, max_shards_per_file=None),)), db_config=DBConfig(db_key='sqlite_usearch', db_config=db_path: !!python/object/apply:etils.epath.gpath.PosixGPath
- /
- workspace
- mojave_data
usearch_cfg:
  dtype: float16
  embedding_dim: 1280
  expansion_add: 256
  expansion_search: 128
  metric_name: IP
), model_config=ModelConfig(model_key='taxonomy_model_tf', embedding_dim=1280, model_config=hop_size_s: 5.0
model_path: ''
sample_rate: 32000
tfhub_version: 8
window_size_s: 5.0
))

In [3]:
#@title Initialize the hoplite database (DB) { vertical-output: true }
global db
db = configs.db_config.load_db()
num_embeddings = db.count_embeddings()

print('Initialized DB located at ', configs.db_config.db_config.db_path)

def drop_and_reload_db(_) -> interface.HopliteDBInterface:
  db_path = epath.Path(configs.db_config.db_config.db_path)
  for fp in db_path.glob('hoplite.sqlite*'):
    fp.unlink()
  (db_path / 'usearch.index').unlink()
  print('\n Deleted previous db at: ', configs.db_config.db_config.db_path)
  db = configs.db_config.load_db()

#@markdown If `drop_existing_db` set to True, when the database already exists and contains embeddings,
#@markdown then those existing embeddings will be erased. You will be prompted to confirm you wish to delete those existing
#@markdown embeddings. If you want to keep existing embeddings in the database, then set to False, which will append the new
#@markdown embeddings to the database.
drop_existing_db = True  #@param {type:'boolean'}

if num_embeddings > 0 and drop_existing_db:
  print('Existing DB contains datasets: ', db.get_dataset_names())
  print('num embeddings: ', num_embeddings)
  print('\n\nClick the button below to confirm you really want to drop the database at ')
  print(f'{configs.db_config.db_config.db_path}\n')
  print(f'This will permanently delete all {num_embeddings} embeddings from the existing database.\n')
  print('If you do NOT want to delete this data, set `drop_existing_db` above to `False` and re-run this cell.\n')

  button = widgets.Button(description='Delete database?')
  button.on_click(drop_and_reload_db)
  display(button)

Initialized DB located at  /workspace/mojave_data


In [4]:
#@title Run the embedding { vertical-output: true }

print(f'Embedding dataset: {audio_glob.dataset_name}')

worker = embed.EmbedWorker(
    audio_sources=configs.audio_sources_config,
    db=db,
    model_config=configs.model_config)

worker.process_all(target_dataset_name=audio_glob.dataset_name)

print('\n\nEmbedding complete, total embeddings: ', db.count_embeddings())

Embedding dataset: mojave_data


/workspace/.venv/lib/python3.11/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
2025-08-07 19:50:19.044961: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 55971840 exceeds 10% of free system memory.
2025-08-07 19:50:19.135630: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 11944960 exceeds 10% of free system memory.
  0%|          | 0/1 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1754596220.376616   12039 service.cc:152] XLA service 0xaaaaf667e8c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754596220.376707   12039 servic



Embedding complete, total embeddings:  4872


In [5]:
#@title Per dataset statistics { vertical-output: true }

for dataset in db.get_dataset_names():
  print(f'\nDataset \'{dataset}\':')
  print('\tnum embeddings: ', db.get_embeddings_by_source(dataset, source_id=None).shape[0])


Dataset 'mojave_data':
	num embeddings:  4872


In [ ]:
#@title Show example embedding search
#@markdown As an example (and to show that the embedding process worked), this
#@markdown selects a single embedding from the database and outputs the embedding ids of the
#@markdown top-K (k = 128) nearest neighbors in the database.

q = db.get_embedding(db.get_one_embedding_id())
%time results, scores = brutalism.brute_search(worker.db, query_embedding=q, search_list_size=128, score_fn=np.dot)
print([int(r.embedding_id) for r in results])

CPU times: user 88.5 ms, sys: 1.61 ms, total: 90.1 ms
Wall time: 91.6 ms
[1, 1049, 804, 1114, 189, 133, 170, 553, 2831, 1977, 127, 2528, 800, 817, 108, 646, 1828, 1888, 547, 91, 2165, 915, 292, 102, 572, 174, 2152, 134, 172, 1823, 799, 284, 2394, 1672, 285, 2104, 118, 2377, 1945, 905, 660, 381, 1545, 1710, 2406, 1740, 534, 575, 2348, 225, 1373, 739, 711, 227, 2160, 1731, 98, 1312, 440, 2027, 1056, 2084, 1753, 2917, 987, 419, 1188, 294, 533, 132, 993, 1369, 2109, 1098, 2076, 816, 1077, 222, 1532, 97, 730, 100, 1363, 107, 1094, 2875, 1573, 1997, 2231, 198, 1686, 1726, 1705, 1989, 226, 1035, 1516, 74, 1165, 2177, 471, 1311, 224, 497, 869, 878, 940, 1968, 153, 1084, 948, 1604, 1783, 2227, 1350, 1412, 1460, 1266, 205, 216, 437, 237, 1119, 1168, 2149, 1934, 580, 2009]


: 